不同关系有多个事实，每个事实有多语言提示 

提示模板取自 TREx_prompts.csv 

提示语言取 Wikidata 中： en nl ja vi zh ko 

关系：

In [ ]:
import json
import pandas as pd
import numpy as np

In [ ]:
# en nl ja vi zh ko 
language_list = ['en', 'nl', 'ja', 'vi', 'zh', 'ko']
relation_list = []
prompt_df = pd.read_csv('./raw/TREx_prompts.csv', index_col=0)

In [ ]:
prompt_df['relation']['P19']

In [ ]:
# facts_list 存储事实，将raw_data的多关系拆解为简单三元组 每个三元组的元素分开
facts_list = []
with open('raw/TREx_multi_rel.txt') as f:
    while  True:
        line  =  f.readline()
        if  not  line:
            break;
        if line is not '':
            line = line.strip().split('\t')
            # print(line)
            for sub in line[-1].split(' '):
                facts_list.append([line[0], line[1], sub])

In [ ]:
count_dic = {}
for fact in facts_list:
    if fact[1] not in count_dic:
        count_dic[fact[1]] = 1
    else:
        count_dic[fact[1]] = count_dic[fact[1]] + 1
count_dic

In [ ]:
# entities_dic 存储实体及不同语言的表示方式 使用字典嵌套字典
entities_dic = {}
with open('raw/TREx_unicode_escape.txt') as f:
    while  True:
        line  =  f.readline()
        if  not  line:
            break;
        id = line.strip().split('\t')[0]
        entities_dic[id] = {}
        for item in line.strip().split('\t'):
            language = item.split('@')[-1]
            if language in language_list:
                entities_dic[id][language] = item.split('@')[0].strip('"')

In [ ]:
with open('../PARAREL/data_all.json') as f:
    data_demo = json.load(f)
    
data_demo[0]

In [58]:
data_all_list = []
data_allbags_dic = {}
for trip in facts_list:
    # 对某一知识构建不同语言的表述
    if np.random.rand() > 0.5:
        # 随机选择 mask 主语还是宾语
        
        fact = []
        for language in language_list:
            if language in entities_dic[trip[0]] and language in entities_dic[trip[2]]:
                data = [
                    prompt_df[language][trip[1]].replace('[X]', entities_dic[trip[0]][language]).replace('[Y]', '[MASK]'),
                    entities_dic[trip[2]][language],
                    trip[1] + f"({prompt_df['relation'][trip[1]]})"
                ]
                fact.append(data)
        if len(fact) > 3:        
            if trip[1] in data_allbags_dic:
                data_allbags_dic[trip[1]].append(fact)
            else:
                data_allbags_dic[trip[1]] = [fact, ]
            data_all_list.append(fact)
        
        
    else:
        fact = []
        for language in language_list:
            if language in entities_dic[trip[0]] and language in entities_dic[trip[2]]:
                data = [
                    prompt_df[language][trip[1]].replace('[Y]', entities_dic[trip[2]][language]).replace('[X]', '[MASK]'),
                    entities_dic[trip[0]][language],
                    trip[1] + f"({prompt_df['relation'][trip[1]]})"
                ]
                fact.append(data)
        if len(fact) > 3:
            # if len(data_all_list) > 2000:
            #     print(fact)
            if trip[1] in data_allbags_dic:
                data_allbags_dic[trip[1]].append(fact)
            else:
                data_allbags_dic[trip[1]] = [fact, ]
            data_all_list.append(fact)
 

In [59]:
len(data_all_list)

25896

In [60]:
with open("data_all.json","w") as f:
    json.dump(data_all_list, f, ensure_ascii=False)

with open("data_allbags.json","w") as f:
    json.dump(data_allbags_dic, f, ensure_ascii=False)

In [61]:
with open('data_allbags.json') as f:
    data = json.load(f)
data.keys()

dict_keys(['P27', 'P31', 'P39', 'P47', 'P101', 'P106', 'P108', 'P178', 'P190', 'P463', 'P527', 'P530', 'P937', 'P1001', 'P1303', 'P1412'])

In [62]:
for key in data.keys():
    print(f"{key}\t{len(data[key])}")

P27	615
P31	714
P39	1603
P47	2684
P101	745
P106	1729
P108	323
P178	419
P190	5476
P463	1127
P527	1637
P530	5926
P937	837
P1001	447
P1303	894
P1412	720
